# --- Day 11: Monkey in the Middle ---

In [7]:
day = 11

## Import data and store as list of list with int type
with open(f'day-{str(day)}-input.txt', 'r') as f:
    data = [i for i in f.read().split("\n\n")]


## Get data into list of lists with each element a line in the monkey instruction, get rid of tabs as it will just make things annoying 
monkeys = []
for i in data:
    monkeys.append([z.strip() for z in i.split('\n')])

In [8]:
## define monkey class!

class Monkey:
    def __init__(self, number, startingItems, monkeyOper, monkeyTest):
        self.number = number
        self.startingItems = startingItems
        self.monkeyOper = monkeyOper
        self.monkeyTest = monkeyTest
        self.numInspections = 0
    
    def __print__(self):
        print(f"""
        Monkey {self.number} has the starting items as follows: {self.startingItems}. \n
        Monkey {self.number} has the following operation {self.monkeyOper} with the follow test values {self.monkeyTest}
            """)

    def splitOperation(self):
        return self.monkeyOper[1],self.monkeyOper[2]

    def splitTest(self):
        return self.monkeyTest[0],self.monkeyTest[1],self.monkeyTest[2]


## Function to set intital monkeys as class object
def makeMonkeys(input):
    monkeyObjs = []

    for idx, monkey in enumerate(monkeys):
        ## Monkey number
        number = idx

        ## Starting Items
        startingItems = monkey[1].replace("Starting items: ", "")
        startingItems = [int(i) for i in startingItems.split(",")]

        ## Monkey Operation
        monkeyOper = monkey[2].replace("Operation: new = ", "")
        monkeyOper = [i for i in monkeyOper.split()]
        
        ## Monkey Test
        monkeyTest = int(monkey[3].replace("Test: divisible by ", ""))
        monkeyTrue = int(monkey[4][-1])
        monkeyFalse = int(monkey[5][-1])

        ## Append parsed values to monkeyObjs list and store as Monkey Object.
        monkeyObjs.append(Monkey(
            number = number,
            startingItems = startingItems,
            monkeyOper = monkeyOper,
            monkeyTest = (monkeyTest,monkeyTrue,monkeyFalse)
        ))
    return monkeyObjs

## Helper function for determining worry level given a monkeys operations

def determineWorry(monkey,item,newMod):
    operation, right = monkey.splitOperation()

    if right == "old":
        if operation == "+" or operation == "-":
            ret =  item + item
        elif operation == "*":
            ret =  item * item
    elif right != "old":
        if operation == "+" or operation == "-":
            ret =  item + int(right) 
        elif operation == "*":
            ret =  item * int(right)
    
    return ret % newMod


## Run the simulation!

def monkeySimulation(monkeyObjs, rounds, newMod):
    ## Each round
    for _ in range(rounds):
        ## Each monkey obj
        for monkey in monkeyObjs :
            ## pull out the split test
            mod, trueMonkey, falseMonkey = monkey.splitTest()

            ## Each item in starting Items
            for item in monkey.startingItems:
                ## Determine worry level
                item = determineWorry(monkey, item, newMod)
                
                ## Determine who to throw to if modular expression is true
                if item % mod == 0:
                    ## add item to the tru monkey
                    monkeyObjs[trueMonkey].startingItems.append(item)
                else:
                    ## add item to the false monkey
                    monkeyObjs[falseMonkey].startingItems.append(item)
                
                ## Add one more inspection to the monkey!
                monkey.numInspections += 1
            
        ## for the particular monkey we have now distributed all the items so the startingItems attribute can be reset
            monkey.startingItems = [] 
    
    return monkeyObjs


In [10]:
## Task 1 
# Figure out which monkeys to chase by counting how many items they inspect over 20 rounds. 
# What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?

## Make Monkeys!
monkeyObjs = makeMonkeys(monkeys)

## Disclaimer, I've had a someone with a maths phd try to explain this to me, i dont understand, i would prefer to let it run for a few months
newMod = 1
for monkey in monkeyObjs:
    newMod *= monkey.monkeyTest[0]

## Number of monkeys!
numMonkeys = len(monkeyObjs)

## Run simulation
monkeyObjs = monkeySimulation(monkeyObjs, 10_000, newMod)

monkeyInspections = sorted([monkey.numInspections for monkey in monkeyObjs],reverse=True)

monkeyInspections[1] * monkeyInspections[0]

30616425600